##Preparativos

In [ ]:
!pip install --q wfdb

In [ ]:
import wfdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dask import delayed
from numba import jit
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Crear Dataframe

Para esta prueba solo se van a importar datos de la primera derivada, sin razón en particular.

In [ ]:
def recover(path,derivative=0):
  #La función recover devuelve un array para la derivada indicada. Las derivadas estan 0 indexed.
  record = wfdb.rdsamp(path, channels=[derivative])
  record = record[0].T #Por ahora se toma solo los datos, no la metainfo
  return record

@jit #Prueba con numba para mayor velocidad
def stick(a,b):
  a = np.concatenate((a,b),axis=0)
  return a

In [ ]:
@delayed  
def create_array(Arreglo, start_point, derivada=0):

  for i in range(start_point, start_point+1000):
    full_number = (5 - len(str(i)))*'0'+str(i)
    start_point = (5 - len(str(start_point)))*'0'+str(start_point)
    
    if (int(full_number)!=0) and (int(full_number)<21838): #Malas practicas?
      query = '/content/drive/MyDrive/ISB/Datos/records100/{}/{}_lr'.format(start_point,full_number)
      record = recover(query,derivada)
      Arreglo = stick(Arreglo,record)
    
  return Arreglo


Algoritmo de extracción

In [ ]:
fs=100
tiempo = np.arange(1000)*(1/fs) #Tiempo, La frecuencia de sampleo es 100, también se podría haber trabajado con los datos de frecuencia de sampleo  en 500.
#Me parecen muchos datos (1000 vs 5000) a mi, por lo que no lo considero
tiempo = tiempo.reshape(1,-1)
derivada = 0 # 0 equivale a la primera derivada, esta indexada en modo python

In [ ]:
DELAYED_VSTACK = delayed(np.vstack)

In [ ]:
#Inicializar arreglos
lista_de_arreglos = [np.zeros((1,1000)) for i in range(22)]

In [ ]:
#PROGRAMACIÓN EN PARALELO
#CUIDADO CON CORRER
for i,k in enumerate(lista_de_arreglos):
  lista_de_arreglos[i] = create_array(k,i*1000)

final_array = DELAYED_VSTACK(lista_de_arreglos).compute()

In [ ]:
'''
OPERACIÓN SECUENCIALN NO CORRER:

start = '00000'

for i in range(1,21838):#Hay 21837 datos, range termina en uno menos
  full_number = (5 - len(str(i)))*'0'+str(i)

  if i%1000 == 0:
    start = (5 - len(str(i)))*'0'+str(i) #Autocorrección del Start

  query = '/content/drive/MyDrive/ISB/Datos/records100/{}/{}_lr'.format(start,full_number)
  record = recover(query,derivada)
  #datos = np.concatenate((datos,record),axis=0)
  datos = stick(datos,record)
  print("Archivo N°: ",i)
'''

'\nOPERACIÓN SECUENCIALN NO CORRER:\n\nstart = \'00000\'\n\nfor i in range(1,21838):#Hay 21837 datos, range termina en uno menos\n  full_number = (5 - len(str(i)))*\'0\'+str(i)\n\n  if i%1000 == 0:\n    start = (5 - len(str(i)))*\'0\'+str(i) #Autocorrección del Start\n\n  query = \'/content/drive/MyDrive/ISB/Datos/records100/{}/{}_lr\'.format(start,full_number)\n  record = recover(query,derivada)\n  #datos = np.concatenate((datos,record),axis=0)\n  datos = stick(datos,record)\n  print("Archivo N°: ",i)\n'

Conversión a pandas y csv

In [ ]:
df = pd.DataFrame(final_array,columns=tiempo[0])
#Se obtienen los datos que no son 0 (Pertenecientes a la inicialización con np.zeros)
df=df[df.sum(axis=1)!=0]
#Se tiene una serie temporal donde las columnas equivalen al tiempo

In [ ]:
df.head()

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,9.90,9.91,9.92,9.93,9.94,9.95,9.96,9.97,9.98,9.99
1,-0.119,-0.116,-0.120,-0.117,-0.103,-0.097,-0.119,-0.096,-0.048,-0.037,...,0.198,0.194,0.115,0.107,0.107,0.106,0.090,0.069,0.086,0.022
2,0.004,-0.020,-0.053,-0.056,-0.062,-0.065,-0.061,-0.061,-0.064,-0.061,...,-0.035,-0.045,0.004,0.044,0.507,0.554,0.316,0.121,-0.326,-0.348
3,-0.029,-0.035,-0.054,-0.078,-0.088,-0.022,0.346,0.784,0.426,-0.076,...,-0.040,-0.051,-0.026,-0.032,-0.052,-0.039,-0.034,-0.029,-0.048,-0.049
4,-0.054,-0.053,-0.063,-0.060,-0.050,-0.054,-0.059,-0.058,-0.054,-0.051,...,-0.312,-0.511,-0.280,-0.076,-0.012,0.001,-0.003,0.026,0.026,0.028
5,-0.034,-0.038,-0.057,-0.066,-0.080,-0.085,-0.058,-0.061,-0.068,-0.065,...,0.005,0.001,0.003,0.013,0.018,-0.001,0.007,0.000,-0.003,-0.012


Se exporta a csv para futuros usos

In [ ]:
df.to_csv('DatosDerivada{}.csv'.format(derivada+1),header=True,index=False)